In [1]:
#The point of this notebook is to aquire and analyze the performance of opossums in the reaching and grasping task.

#We will apply a bath loading pipeline developed elsewhere

#We will extract performance and limb selecton data

#We will apply appropriate statistics to analyze this data.

In [2]:
#Packages

import re
import pandas as pd
import os
import numpy as np
import matplotlib.pyplot as plt

#Classes
from preprocessing.classes import BatchLoader
batchLoader = BatchLoader('tester')



In [3]:
#Bathload and arrange


#Use Bathloader to fetch the files
path = '/Users/carlospineda/Desktop/TEMP Reaching Results for Code Dev/performance/'
files = batchLoader.getFile(path)
performance_multi = pd.DataFrame()
performancePerFile = pd.DataFrame()
for file in files:
    if not file.startswith('.'):
        csv = pd.read_csv(path + file)
        csv = csv[14:].reset_index(drop = True)
        #print(csv)
        csv.columns = csv.iloc[0]

        #Calculate percent performance as ((1*contact) + (2*Sweep/Drop) + (3*success))/(3*TotalReachAttempts)
        #print(len(csv))
        #Performance for this file

        #Initialize variables
        miss = 0
        contact = 0
        SweepDrop = 0
        success = 0

        #figure out what behaviors were coded
        uniques = csv['Behavior'].unique()

        #get number of reaches
        TotalReachAttempts = len(csv)

        #get number of occurrances per behavior
        numContacts = csv['Behavior'].value_counts()

        #In case any are missing, use if statements.
        if 'miss' in uniques:
            miss = numContacts['miss']
        else:
            print('no misses in this' +''+ file)

        if 'contact' in uniques:
            contact = numContacts['contact']
        else:
            print('no contact in this'+ file)

        if 'Sweep/Drop' in uniques:
            SweepDrop = numContacts['SweepDrop']
        else:
            print('no sweeps in this' + file)

        if 'success' in uniques:
            success = numContacts['success']
        else:
            print('no successes in this' + file)


        #CALCULATE PERFORAMANCE
        #performance = (((1*contact) + (2*SweepDrop) + (3*success))/(3*(len(csv)-1)))
        performance = (success)/(len(csv))
        #print(performance)
        #ADD COLUMNS
            #This should add the column we want.
        performancePerFile['Performance']= np.nan
        performancePerFile['Condition'] = np.nan
        performancePerFile['Light'] = np.nan
        performancePerFile['Whisk'] = np.nan
        performancePerFile['Smell'] = np.nan

        #add the performance measure to the performance column
        performancePerFile['Performance'] = performance

        #Use regex to extract file text from the title.
        animalCond_blindBool = re.search(r'_blind_', str(file))
        animalCond_contBool = re.search(r'_control_', str(file))
        #light condition
        trialCond_darkBool = re.search(r'_dark_',  str(file))
        trialCond_lightBool = re.search(r'_light_',  str(file))
        #whisker condition
        trialCond_trimBool = re.search(r'_whisk_',  str(file))
        trialCond_untrimBool = re.search(r'_unwhisk_',  str(file))
        #Smell condition
        trialCond_smellBool = re.search(r'_smell_', str(file))
        trialCond_unsmellBool = re.search(r'_unsmell_', str(file))


        #Add condition of the animal (blind/control) to the condition column.
        if animalCond_blindBool:
            performancePerFile['Condition'] = list(['blind'])
        elif animalCond_contBool:
            performancePerFile['Condition'] = list(['control'])
        #light
        if trialCond_darkBool:
            performancePerFile['Light'] = list(['dark'])
        elif trialCond_lightBool:
            performancePerFile['Light'] = list(['light'])
        #whisker trim
        if trialCond_trimBool:
            performancePerFile['Whisk'] = list(['trim'])
        elif trialCond_untrimBool:
            performancePerFile['Whisk'] = list(['untrim'])
        #anosmia condition
        if trialCond_smellBool:
            performancePerFile['Smell'] = list(['smell'])
        elif animalCond_contBool:
            performancePerFile['Smell'] = list(['unsmell'])


        performance_multi = pd.concat([performance_multi, performancePerFile])
#print(performance_multi)
print(performance_multi)

no misses in thisbehavior_shape4_cam3_21055_blind__light_whisk_smell_121821.csv
no sweeps in thisbehavior_shape4_cam3_21055_blind__light_whisk_smell_121821.csv
no sweeps in thisbehavior_shape4_cam2_11090_blind_light_whisk_smell_122421.csv
no sweeps in thisbehavior_test3_cam2_11094_control_dark_whisk_smell_031022.csv
no sweeps in thisbehavior_test3_cam3_11093_blind_dark_whisk_smell_031022.csv
no sweeps in thisbehavior_test3_cam2_11093_blind_dark_whisk_smell_031022.csv
no contact in thisbehavior_test3_cam3_11094_control_dark_whisk_smell_031022.csv
no sweeps in thisbehavior_test3_cam3_11094_control_dark_whisk_smell_031022.csv
no sweeps in thisbehavior_test3_cam1_11092_blind_dark_whisk_smell_031022.csv
no sweeps in thisbehavior_test3_cam3_11092_blind_dark_whisk_smell_031022.csv
no sweeps in thisbehaviour_test3_cam1_11093_blind_dark_whisk_smell_031022.csv
no sweeps in thisbehavior_test3_cam2_11092_blind_dark_whisk_smell_031022.csv
no sweeps in thisbehavior_shape4_cam2_11093_blind_light_whis

In [4]:
#We want to find the percent success rate for each condition

performanceByGroupMean = performance_multi.groupby(['Condition', 'Light']).mean()
performanceByGroupSTD = performance_multi.groupby(['Condition', 'Light']).std()
print(performanceByGroupMean)
print(performanceByGroupSTD)


                 Performance
Condition Light             
blind     dark      0.551504
          light     0.476923
control   dark      0.668369
          light     0.641414
                 Performance
Condition Light             
blind     dark      0.177992
          light     0.277149
control   dark      0.187728
          light     0.289998


In [5]:
#Simple visualization